In [1]:
import requests, pickle, os , re, gspread
from bs4 import BeautifulSoup
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from random import randint
from time import sleep
from telethon import TelegramClient, sync, events

In [2]:
def get_spreadsheets(file_name):
    file = open('{}'.format(file_name), 'r')
    content = file.read()
    spreadsheets_with_None = content.split('\n')
    spreadsheets = [x.split('/')[-1] for x in spreadsheets_with_None if x]
    return spreadsheets

def write_success_report(spreadsheet, row, login, title, state, date, clicks, balance):
    spreadsheet.update_cell(row, 1, login)
    spreadsheet.update_cell(row, 2, title)
    spreadsheet.update_cell(row, 3, state)
    spreadsheet.update_cell(row, 4, date)
    spreadsheet.update_cell(row, 5, clicks)
    spreadsheet.update_cell(row, 6, balance)
    spreadsheet.update_cell(row, 7, str(datetime.now()))
    spreadsheet.update_cell(row, 8, "")
    
def write_error_report(spreadsheet, row, login, error):
    spreadsheet.update_cell(row, 1, login)
    spreadsheet.update_cell(row, 8, error)
    
def clear_row(spreadsheet, row):
    spreadsheet.update_cell(row, 1, "")
    spreadsheet.update_cell(row, 2, "")
    spreadsheet.update_cell(row, 3, "")
    spreadsheet.update_cell(row, 4, "")
    spreadsheet.update_cell(row, 5, "")
    spreadsheet.update_cell(row, 6, "")
    spreadsheet.update_cell(row, 7, "")
    spreadsheet.update_cell(row, 8, "")

def get_bot_credentials(filename):
    file = open('auth/{}'.format(filename), 'r')
    content = file.read()
    credentials_with_None = content.split('\n')
    credentials = [x for x in credentials_with_None if x]
    return {'api_id': int(credentials[0]), 'api_hash': credentials[1], 'bot_token': credentials[2]}

def get_id_from_string(str):
    try:
        id = int(str)
        return id
    except:
        pass

def get_bot_admins_list(filename):
    file = open(filename, 'r')
    content = file.read()
    list_with_None = list(map(get_id_from_string, content.split('\n')))
    final_list = [x for x in list_with_None if x]
    return final_list

In [3]:
# Use your own values here
creds = get_bot_credentials('credentials.txt')

telegram_client = TelegramClient('bot_session', creds['api_id'], creds['api_hash'])
telegram_client.start(bot_token=creds['bot_token'])

In [4]:
login_col, passwd_col, threshold_col, proxyIP_col, proxyLogin_col, proxyPass_col, userAgent_col = 2, 4, 6, 8, 10, 12, 14

def run_monitor():
    # use creds to create a client to interact with the Google Drive API
    # (https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html)
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('auth/client_secret.json', scope)
    google_client = gspread.authorize(creds)
    for spreadsheet_name in get_spreadsheets('data/spreadsheets.txt'):
        source_row = 1
        report_row = 2
        source_spreadsheet = google_client.open(spreadsheet_name).sheet1
        try: 
            report_spreadsheet = google_client.open(spreadsheet_name + '_report').sheet1
        except:
            report_spreadsheet = None
        if report_spreadsheet:
            while len(source_spreadsheet.cell(source_row, login_col).value) > 0:
                try:
                    login = source_spreadsheet.cell(source_row, login_col).value
                    passwd = source_spreadsheet.cell(source_row, passwd_col).value
                    
                    try:
                        threshold = float(source_spreadsheet.cell(source_row, threshold_col).value)
                    except:
                        threshold = 0
                    
                    proxyIP = source_spreadsheet.cell(source_row, proxyIP_col).value
                    proxyLogin = source_spreadsheet.cell(source_row, proxyLogin_col).value
                    proxyPass = source_spreadsheet.cell(source_row, proxyPass_col).value
                    userAgent = source_spreadsheet.cell(source_row, userAgent_col).value
                    proxies = {
                        'http': f'http://{proxyLogin}:{proxyPass}@{proxyIP}',
                        'https': f'https://{proxyLogin}:{proxyPass}@{proxyIP}',
                    }

                    session_filename = f'{login.split("@")[0] + "@" + login.split("@")[1].split(".")[0]}.pkl'
                    session = requests.Session()
                    session.headers['User-Agent'] = userAgent
                    if os.path.isfile(f'./yandex_sessions/{spreadsheet_name}_report/{session_filename}'):
                        with open(f'./yandex_sessions/{spreadsheet_name}_report/{session_filename}', 'rb') as f:
                            session = pickle.load(f)
                    else:
                        url = 'https://passport.yandex.ru/passport?mode=auth'
                        data = {'login': login, 'passwd': passwd}
                        session.get(url)
                        r = session.post(url, data=data, proxies=proxies)
                        if 'yandex_login' in session.cookies.get_dict():
                            with open(f'./yandex_sessions/{spreadsheet_name}_report/{session_filename}', 'wb') as f:
                                pickle.dump(session, f)
                            
                    response = session.get('https://direct.yandex.ru/registered/main.pl?cmd=showCamps', proxies=proxies)
                    html = BeautifulSoup(response.text, 'html.parser')

                    balance = html.find("div", {"class":"b-wallet-link__title-sum b-wallet-link__valign-middle"}).text
                    if float(balance[:-5].replace('\xa0', '')) <= threshold:
                        for user_id in get_bot_admins_list('data/admins.txt'):
                            telegram_client.send_message(user_id, f'Счёт этого аккаунта {login} ниже {threshold} руб. значения')

                    campaigns = html.findAll("tr", {"class":"b-campaigns-list-item"})
                    for campaign in campaigns:
                        title = campaign.find(("h3", {"class":"b-campaigns-list-item__title"})).text
                        state = campaign.find("div", {"class":"b-campaigns-list-item__state"}).text
                        date = campaign.find("div", {"class":"b-campaigns-list-item__date"}).text
                        clicks = campaign.find("div", {"class":"b-list-numeric b-list-numeric_type_clicks"}).text

                        write_success_report(report_spreadsheet, report_row,
                                             login, title, state, date, clicks, balance)
                        report_row += 1
                        sleep(10)
                except Exception as e:
                    write_error_report(report_spreadsheet, report_row, login, e.__str__())
                    report_row += 1
                    sleep(10)
                finally:
                    source_row += 1
            while len(report_spreadsheet.cell(report_row, 1).value) > 0:
                clear_row(report_spreadsheet, report_row)
                report_row += 1
                sleep(10)

In [5]:
while True:
    run_monitor()
    sleep(randint(300, 1000))

APIError: {
  "error": {
    "code": 500,
    "message": "Internal error encountered.",
    "status": "INTERNAL"
  }
}
